# [PEP 3119 -- Introducing Abstract Base Classes](http://www.python.org/dev/peps/pep-3119/)

In [16]:
from abc import ABCMeta

class MyABC(object):
    __metaclass__ = ABCMeta
    def valami():
        print "I'm someting"

MyABC.register(tuple)

In [17]:
issubclass(tuple, MyABC)

True

In [18]:
isinstance((), MyABC)

True

In [19]:
class MyClass(MyABC):
    pass

In [20]:
issubclass(MyClass, MyABC)

True

In [21]:
isinstance(MyClass(), MyABC)

True

In [22]:
issubclass(tuple, MyClass)

False

In [23]:
isinstance((), MyClass)

False

In [24]:
MyClass.register(list)

In [25]:
issubclass(list, MyClass)

True

In [26]:
issubclass(list, MyABC)

True

In [27]:
isinstance([], MyClass)

True

In [28]:
class AnotherClass(object):
    __metaclass__ = ABCMeta

AnotherClass.register(basestring)
MyClass.register(AnotherClass)

In [29]:
issubclass(str, MyABC)

True

@abstractmethod decorator

In [30]:
from abc import ABCMeta, abstractmethod

class A(object):
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def foo(self): pass

A()

TypeError: Can't instantiate abstract class A with abstract methods foo

In [31]:
class B(A):
    pass

B()

TypeError: Can't instantiate abstract class B with abstract methods foo

In [32]:
class  C(A):
    def foo(self): 
        print 42

C()

## ABstract base classes in collections module

In [33]:
from collections import Sequence, MutableSequence

In [34]:
isinstance([], Sequence)

True

In [35]:
issubclass(list, Sequence)

True

In [36]:
issubclass(list, MutableSequence)

True

In [37]:
isinstance((), Sequence)

True

In [38]:
not issubclass(tuple, MutableSequence)

True

In [39]:
isinstance("", Sequence)

True

In [40]:
issubclass(str, Sequence)

True

## [Python: How to “perfectly” override a dict](http://stackoverflow.com/questions/3387691/python-how-to-perfectly-override-a-dict)

You can write an object that behaves like a dict quite easily with ABCs (Abstract Base Classes) from the collections module. It even tells you if you missed a method, so below is the minimal version that shuts the ABC up.

In [41]:
import collections


class TransformedDict(collections.MutableMapping):
    """A dictionary that applies an arbitrary key-altering
       function before accessing the keys"""

    def __init__(self, *args, **kwargs):
        self.store = dict()
        self.update(dict(*args, **kwargs))  # use the free update to set keys

    def __getitem__(self, key):
        return self.store[self.__keytransform__(key)]

    def __setitem__(self, key, value):
        self.store[self.__keytransform__(key)] = value

    def __delitem__(self, key):
        del self.store[self.__keytransform__(key)]

    def __iter__(self):
        return iter(self.store)

    def __len__(self):
        return len(self.store)

    def __keytransform__(self, key):
        return key

You get a few free methods from the ABC:

In [42]:
class MyTransformedDict(TransformedDict):

    def __keytransform__(self, key):
        return key.lower()


s = MyTransformedDict([('Test', 'test')])

assert s.get('TEST') is s['test']   # free get
assert 'TeSt' in s                  # free __contains__
                                    # free setdefault, __eq__, and so on

import pickle
assert pickle.loads(pickle.dumps(s)) == s
                                    # works too since we just use a normal dict

In [43]:
class Valami(object):
    valami = 5
Valami.__dict__

<dictproxy {'__dict__': <attribute '__dict__' of 'Valami' objects>,
 '__doc__': None,
 '__module__': '__main__',
 '__weakref__': <attribute '__weakref__' of 'Valami' objects>,
 'valami': 5}>

# Property

In [44]:
from abc import ABCMeta, abstractproperty
class AbstractValami(object):
    __metaclass__ = ABCMeta
    
    @abstractproperty
    def nickname(self):
        pass

In [45]:
class Valami(AbstractValami):
    def __init__(self):
        self.nickname = "semmi"
        pass

In [46]:
Valami()

TypeError: Can't instantiate abstract class Valami with abstract methods nickname

# Iterator is simple

In [47]:
class MyIterator(object):
    def __iter__(self):
        return self
    def next(self):
        return 42

In [48]:
from collections import Iterator
isinstance(MyIterator(), Iterator)

True

# Introspectable

In [49]:
import collections
collections.Sequence.__abstractmethods__

frozenset({'__getitem__', '__len__'})

In [50]:
collections.Mapping.__abstractmethods__

frozenset({'__getitem__', '__iter__', '__len__'})

In [51]:
collections.MutableMapping.__abstractmethods__

frozenset({'__delitem__', '__getitem__', '__iter__', '__len__', '__setitem__'})